# <b><span style='color:#F1A424'>|</span> HMS Wavenet All Channels</b><a class='anchor' id='configuration'></a> [↑](#top) 

In [1]:
import gc
import math
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import os
import pandas as pd
import random
import time
import torch
import torch.nn as nn


from glob import glob
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from typing import Dict, List

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using', torch.cuda.device_count(), 'GPU(s)')
!mkdir models5

Using 1 GPU(s)
mkdir: cannot create directory ‘models5’: File exists


# <b><span style='color:#F1A424'>|</span> Configuration</b><a class='anchor' id='configuration'></a> [↑](#top) 

***

In [2]:
class config:
    AMP = True
    BATCH_SIZE_TRAIN = 64#32
    BATCH_SIZE_VALID = 64#32
    EPOCHS = 15
    FOLDS = 4
    GRADIENT_ACCUMULATION_STEPS = 1
    MAX_GRAD_NORM = 1e7
    NUM_WORKERS = 1 #multiprocessing.cpu_count()
    PRINT_FREQ = 50
    SEED = 30#20
    TRAIN_FULL_DATA = False
    VISUALIZE = True
    WEIGHT_DECAY = 0.01
    
    
class paths:
    OUTPUT_DIR = "/kaggle/working/"
    TRAIN_CSV = "../shared_data/hms_data/raw_data/train.csv"
    TRAIN_EEGS = "../shared_data/hms_data/raw_data/train_eegs/"

# <b><span style='color:#F1A424'>|</span> Utils</b><a class='anchor' id='utils'></a> [↑](#top) 

***

Utility functions.

In [3]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s: float):
    "Convert to minutes."
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since: float, percent: float):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))




def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed) 
    
    
def sep():
    print("-"*100)

    
target_preds = [x + "_pred" for x in ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']]
label_to_num = {'Seizure': 0, 'LPD': 1, 'GPD': 2, 'LRDA': 3, 'GRDA': 4, 'Other':5}
num_to_label = {v: k for k, v in label_to_num.items()}
seed_everything(config.SEED)

# <b><span style='color:#F1A424'>|</span> Load Data</b><a class='anchor' id='load_data'></a> [↑](#top) 

***

Load the competition's data.

In [4]:
train_df = pd.read_csv(paths.TRAIN_CSV)
label_cols = train_df.columns[-6:]
print(f"Train cataframe shape is: {train_df.shape}")
print(f"Labels: {list(label_cols)}")
train_df.head()

Train cataframe shape is: (106800, 15)
Labels: ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']


,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


### <b><span style='color:#F1A424'>Read one EEG parquet</span></b>

All of the EEG data (for both train and test) was collected at a frequency of 200 samples per second,

Each EEG parquet results in a dataframe with `seconds` rows and 20 columns.

- EEG features are: `['Fp1', 'F3', 'C3', 'P3', 'F7', 'T3', 'T5', 'O1', 'Fz', 'Cz', 'Pz', 'Fp2', 'F4', 'C4', 'P4', 'F8', 'T4', 'T6', 'O2', 'EKG']`
- We will use these features: `['Fp1','T3','C3','O1','Fp2','C4','T4','O2']`



In [5]:
#eeg_df = pd.read_parquet(paths.TRAIN_EEGS + "100261680.parquet")
#eeg_features = eeg_df.columns
#print(f'There are {len(eeg_features)} raw eeg features')
#print(list(eeg_features))
eeg_features = ['Fp1', 'F3', 'C3', 'P3', 'F7', 'T3', 'T5', 'O1', 'Fz', 'Cz', 'Pz', 'Fp2', 'F4', 'C4', 'P4', 'F8', 'T4', 'T6', 'O2', 'EKG']
#['Fp1','T3','C3','O1','Fp2','C4','T4','O2']
feature_to_index = {x:y for x,y in zip(eeg_features, range(len(eeg_features)))}

### <b><span style='color:#F1A424'>Read all EEG parquets</span></b>

In [ ]:
### read in each npy file

FILT_EEG_PATH = '../shared_data/hms_data/all_channels_filt_eegs/all_channels_filt_eegs' # all_filt_eegs/'
#all_eegs = np.empty((0, 10000, 20))  # this will hold all the filtered eegs
all_eegs = dict() # this will hold all the filtered eegs

for i in range (1,172):
        #print(i)
        this_dict = np.load(FILT_EEG_PATH + '{:03d}'.format(i) + '.npy', allow_pickle=True).item()
        all_eegs.update(this_dict)

print(len(all_eegs))


In [ ]:
#labels = pd.DataFrame(np.load('../shared_data/hms_data/eeg_filt_labels.npy',allow_pickle=True))

In [ ]:
train_df

# <b><span style='color:#F1A424'>|</span> Data pre-processing</b><a class='anchor' id='preprocessing'></a> [↑](#top) 

***

In [ ]:
eeg_ids = train_df.eeg_id.unique()
len(eeg_ids)

In [ ]:
df = pd.read_csv(paths.TRAIN_CSV)
df = df[df['eeg_id'].isin(all_eegs.keys())]
label_cols = df.columns[-6:]

train_df = df.groupby('eeg_id')[['patient_id']].agg('first')
aux = df.groupby('eeg_id')[label_cols].agg('sum') 

for label in label_cols:
    train_df[label] = aux[label].values
    
y_data = train_df[label_cols].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train_df[label_cols] = y_data

aux = df.groupby('eeg_id')[['expert_consensus']].agg('first')
train_df['target'] = aux

del aux, y_data, df

train_df = train_df.reset_index()
train_df = train_df.loc[train_df.eeg_id.isin(eeg_ids)]
print(f"Train dataframe with unique eeg_id has shape: {train_df.shape}")
train_df.head()

In [ ]:
train_df

# <b><span style='color:#F1A424'>|</span> Validation</b><a class='anchor' id='validation'></a> [↑](#top) 

***

We train using `GroupKFold` on `patient_id`.

In [ ]:
from sklearn.model_selection import KFold, StratifiedGroupKFold


gkf = StratifiedGroupKFold(n_splits=config.FOLDS,shuffle=True,random_state=20)#31)#42)
for fold, (train_index, valid_index) in enumerate(gkf.split(train_df, train_df.target, train_df.patient_id)):
    train_df.loc[valid_index, "fold"] = int(fold)
    
display(train_df.groupby('fold').size()), sep()
display(train_df.head())

# <b><span style='color:#F1A424'>|</span> Dataset</b><a class='anchor' id='dataset'></a> [↑](#top) 

***

In [ ]:
class CustomDataset(Dataset):
    def __init__(
        self, df: pd.DataFrame, config, mode: str = 'train',
        eegs: Dict[int, np.ndarray] = all_eegs, downsample: int = 5
    ): 
        self.df = df
        self.config = config
        self.batch_size = self.config.BATCH_SIZE_TRAIN
        self.mode = mode
        self.eegs = eegs
        self.downsample = downsample
        
    def __len__(self):
        """
        Length of dataset.
        """
        return len(self.df)
        
    def __getitem__(self, index):
        """
        Get one item.
        """
        X, y = self.__data_generation(index)
        X = X[::self.downsample,:]
        output = {
            "X": torch.tensor(X, dtype=torch.float32),
            "y": torch.tensor(y, dtype=torch.float32)
        }
        return output
                        
    def __data_generation(self, index):
        row = self.df.iloc[index]
        X = np.zeros((10_000, 18), dtype='float32')
        y = np.zeros(6, dtype='float32')
        data = self.eegs[row.eeg_id]
        
        # === Feature engineering ===
        
        # Chains:
        
        #['Fp1', 'F3', 'C3', 'P3', 'F7', 'T3', 'T5', 'O1', 'Fz', 'Cz', 'Pz', 'Fp2', 'F4', 'C4', 'P4', 'F8', 'T4', 'T6', 'O2', 'EKG']
        
        
        X[:,0] = data[:,feature_to_index['Fp1']] - data[:,feature_to_index['F7']]
        X[:,1] = data[:,feature_to_index['F7']] - data[:,feature_to_index['T3']]
        X[:,2] = data[:,feature_to_index['T3']] - data[:,feature_to_index['T5']]
        X[:,3] = data[:,feature_to_index['T5']] - data[:,feature_to_index['O1']]
        
        X[:,4] = data[:,feature_to_index['Fp1']] - data[:,feature_to_index['F3']]
        X[:,5] = data[:,feature_to_index['F3']] - data[:,feature_to_index['C3']]
        X[:,6] = data[:,feature_to_index['C3']] - data[:,feature_to_index['P3']]
        X[:,7] = data[:,feature_to_index['P3']] - data[:,feature_to_index['O1']]
        
        X[:,8] = data[:,feature_to_index['Fp2']] - data[:,feature_to_index['F4']]
        X[:,9] = data[:,feature_to_index['F4']] - data[:,feature_to_index['C4']]
        X[:,10] = data[:,feature_to_index['C4']] - data[:,feature_to_index['P4']]
        X[:,11] = data[:,feature_to_index['P4']] - data[:,feature_to_index['O2']]
        
        X[:,12] = data[:,feature_to_index['Fp2']] - data[:,feature_to_index['F8']]
        X[:,13] = data[:,feature_to_index['F8']] - data[:,feature_to_index['T4']]
        X[:,14] = data[:,feature_to_index['T4']] - data[:,feature_to_index['T6']]
        X[:,15] = data[:,feature_to_index['T6']] - data[:,feature_to_index['O2']]
        
        X[:,16] = data[:,feature_to_index['Fz']] - data[:,feature_to_index['Cz']]
        X[:,17] = data[:,feature_to_index['Cz']] - data[:,feature_to_index['Pz']]
        
        
        # Raw EEGs
        #X = data[:,:19]
        # === Standarize ===
        X = np.clip(X,-1024, 1024)
        X = np.nan_to_num(X, nan=0) / 32.0

#         # === Butter Low-pass Filter ===
#         X = butter_lowpass_filter(X)
        
        if self.mode != 'test':
            y = row[label_cols].values.astype(np.float32)
            
        return X, y

# <b><span style='color:#F1A424'>|</span> DataLoader</b><a class='anchor' id='dataloader'></a> [↑](#top) 

***

In [ ]:
train_dataset = CustomDataset(train_df, config, mode="train")
train_loader = DataLoader(
    train_dataset,
    batch_size=config.BATCH_SIZE_TRAIN,
    shuffle=False,
    num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=True
)
output = train_dataset[0]
X, y = output["X"], output["y"]
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

### <b><span style='color:#F1A424'> Visualize DataLoader</span></b>


In [ ]:
if config.VISUALIZE:
    for batch in train_loader:
        X = batch.pop("X")
        y = batch.pop("y")
        for item in range(4):
            plt.figure(figsize=(20,4))
            offset = 0
            for col in range(X.shape[-1]):
                if col != 0:
                    offset -= X[item,:,col].min()
                plt.plot(range(2_000), X[item,:,col]+offset,label=f'feature {col+1}')
                offset += X[item,:,col].max()
            tt = f'{y[col][0]:0.1f}'
            for t in y[col][1:]:
                tt += f', {t:0.1f}'
            plt.title(f'EEG_Id = {eeg_ids[item]}\nTarget = {tt}',size=14)
            plt.legend()
            plt.show()
        break

# <b><span style='color:#F1A424'>|</span> Model</b><a class='anchor' id='model'></a> [↑](#top) 

***

<center><img width = 800 src="https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/Jan-2024/wave-model.png"></center>

In [4]:
class Wave_Block(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, dilation_rates: int, kernel_size: int = 3):
        """
        WaveNet building block.
        :param in_channels: number of input channels.
        :param out_channels: number of output channels.
        :param dilation_rates: how many levels of dilations are used.
        :param kernel_size: size of the convolving kernel.
        """
        super(Wave_Block, self).__init__()
        self.num_rates = dilation_rates
        self.convs = nn.ModuleList()
        self.filter_convs = nn.ModuleList()
        self.gate_convs = nn.ModuleList()
        self.convs.append(nn.Conv1d(in_channels, out_channels, kernel_size=1, bias=True))
        
        dilation_rates = [2 ** i for i in range(dilation_rates)]
        for dilation_rate in dilation_rates:
            self.filter_convs.append(
                nn.Conv1d(out_channels, out_channels, kernel_size=kernel_size,
                          padding=int((dilation_rate*(kernel_size-1))/2), dilation=dilation_rate))
            self.gate_convs.append(
                nn.Conv1d(out_channels, out_channels, kernel_size=kernel_size,
                          padding=int((dilation_rate*(kernel_size-1))/2), dilation=dilation_rate))
            self.convs.append(nn.Conv1d(out_channels, out_channels, kernel_size=1, bias=True))
        
        for i in range(len(self.convs)):
            nn.init.xavier_uniform_(self.convs[i].weight, gain=nn.init.calculate_gain('relu'))
            nn.init.zeros_(self.convs[i].bias)

        for i in range(len(self.filter_convs)):
            nn.init.xavier_uniform_(self.filter_convs[i].weight, gain=nn.init.calculate_gain('relu'))
            nn.init.zeros_(self.filter_convs[i].bias)

        for i in range(len(self.gate_convs)):
            nn.init.xavier_uniform_(self.gate_convs[i].weight, gain=nn.init.calculate_gain('relu'))
            nn.init.zeros_(self.gate_convs[i].bias)

    def forward(self, x):
        x = self.convs[0](x)
        res = x
        for i in range(self.num_rates):
            tanh_out = torch.tanh(self.filter_convs[i](x))
            sigmoid_out = torch.sigmoid(self.gate_convs[i](x))
            x = tanh_out * sigmoid_out
            x = self.convs[i + 1](x) 
            res = res + x
        return res
    
class WaveNet(nn.Module):
    def __init__(self, input_channels: int = 1, kernel_size: int = 3):
        super(WaveNet, self).__init__()
        self.model = nn.Sequential(
                Wave_Block(input_channels, 8, 12, kernel_size),
                Wave_Block(8, 16, 8, kernel_size),
                Wave_Block(16, 32, 4, kernel_size),
                Wave_Block(32, 64, 1, kernel_size) 
        )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.permute(0, 2, 1) 
        output = self.model(x)
        return output


class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.model = WaveNet()
        self.global_avg_pooling = nn.AdaptiveAvgPool1d(1)
        self.dropout = 0.0
        self.head = nn.Sequential(
            nn.Linear(256, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(self.dropout),
            nn.Linear(64, 6)
        )
        
    def forward(self, x: torch.Tensor):
        """
        Forwward pass.
        """
        x1 = self.model(x[:, :, 0:1])
        x1 = self.global_avg_pooling(x1)
        x1 = x1.squeeze()
        x2 = self.model(x[:, :, 1:2])
        x2 = self.global_avg_pooling(x2)
        x2 = x2.squeeze()
        x3 = self.model(x[:, :, 2:3])
        x3 = self.global_avg_pooling(x3)
        x3 = x3.squeeze()
        x4 = self.model(x[:, :, 3:4])
        x4 = self.global_avg_pooling(x4)
        x4 = x4.squeeze()
        z1 = torch.mean(torch.stack([x1, x2, x3, x4]), dim=0)

        x1 = self.model(x[:, :, 4:5])
        x1 = self.global_avg_pooling(x1)
        x1 = x1.squeeze()
        x2 = self.model(x[:, :, 5:6])
        x2 = self.global_avg_pooling(x2)
        x2 = x2.squeeze()
        x3 = self.model(x[:, :, 6:7])
        x3 = self.global_avg_pooling(x3)
        x3 = x3.squeeze()
        x4 = self.model(x[:, :, 7:8])
        x4 = self.global_avg_pooling(x4)
        x4 = x4.squeeze()
        z2 = torch.mean(torch.stack([x1, x2, x3, x4]), dim=0)
        
        x1 = self.model(x[:, :, 8:9])
        x1 = self.global_avg_pooling(x1)
        x1 = x1.squeeze()
        x2 = self.model(x[:, :, 9:10])
        x2 = self.global_avg_pooling(x2)
        x2 = x2.squeeze()
        x3 = self.model(x[:, :, 10:11])
        x3 = self.global_avg_pooling(x3)
        x3 = x3.squeeze()
        x4 = self.model(x[:, :, 11:12])
        x4 = self.global_avg_pooling(x4)
        x4 = x4.squeeze()
        z3 = torch.mean(torch.stack([x1, x2, x3, x4]), dim=0)
        
        x1 = self.model(x[:, :, 12:13])
        x1 = self.global_avg_pooling(x1)
        x1 = x1.squeeze()
        x2 = self.model(x[:, :, 13:14])
        x2 = self.global_avg_pooling(x2)
        x2 = x2.squeeze()
        x3 = self.model(x[:, :, 14:15])
        x3 = self.global_avg_pooling(x3)
        x3 = x3.squeeze()
        x4 = self.model(x[:, :, 15:16])
        x4 = self.global_avg_pooling(x4)
        x4 = x4.squeeze()
        z4 = torch.mean(torch.stack([x1, x2, x3, x4]), dim=0)
        
        x1 = self.model(x[:, :, 16:17])
        x1 = self.global_avg_pooling(x1)
        x1 = x1.squeeze()
        x2 = self.model(x[:, :, 17:18])
        x2 = self.global_avg_pooling(x2)
        x2 = x2.squeeze()
        z5 = torch.mean(torch.stack([x1, x2]), dim=0)
        
        y = torch.cat([z1, z2, z3, z4], dim=1)
        y = self.head(y)
        
        return y

model = CustomModel()
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

Total number of parameters: 98086


# <b><span style='color:#F1A424'>|</span> Scheduler</b><a class='anchor' id='scheduler'></a> [↑](#top) 

***

In [16]:
model = CustomModel()
fold = 3
# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load the state_dict from the saved checkpoint
state_dict_path = f"modelsA6/wavenet_fold_{fold}_best.pth"
state_dict = torch.load(state_dict_path)['model']

# Remove "module." from the keys in the state_dict
new_state_dict = {key.replace("module.", ""): value for key, value in state_dict.items()}

# Load the modified state_dict into the model
model.load_state_dict(new_state_dict)

# Save the modified model state_dict back to the checkpoint
torch.save({'model': model.state_dict(),
            'predictions': torch.load(state_dict_path)['predictions'],
            'best_loss': torch.load(state_dict_path)['best_loss']},
           state_dict_path)

In [9]:
state_dict_path = f"modelsA8/wavenet_fold_1_best.pth"
state_dict = torch.load(state_dict_path)['model']

In [10]:
state_dict

OrderedDict([('module.model.model.0.convs.0.weight',
              tensor([[[-1.0659]],
              
                      [[-0.4965]],
              
                      [[ 0.3991]],
              
                      [[ 0.6779]],
              
                      [[-0.4878]],
              
                      [[ 0.7320]],
              
                      [[ 0.2448]],
              
                      [[ 0.0442]]], device='cuda:0')),
             ('module.model.model.0.convs.0.bias',
              tensor([-0.0284, -0.0069, -0.0057,  0.0006,  0.0031,  0.0127, -0.0197,  0.0041],
                     device='cuda:0')),
             ('module.model.model.0.convs.1.weight',
              tensor([[[ 0.0985],
                       [ 0.6906],
                       [-0.8392],
                       [-0.0186],
                       [-0.1998],
                       [ 0.7256],
                       [-0.3579],
                       [ 0.7902]],
              
               

In [ ]:
from torch.optim.lr_scheduler import OneCycleLR

EPOCHS = config.EPOCHS
BATCHES = len(train_loader)
steps = []
lrs = []
optim_lrs = []
model = CustomModel()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = OneCycleLR(
    optimizer,
    max_lr=1e-3,
    epochs=config.EPOCHS,
    steps_per_epoch=len(train_loader),
    pct_start=0.1,
    anneal_strategy="cos",
    final_div_factor=100,
)
for epoch in range(EPOCHS):
    for batch in range(BATCHES):
        scheduler.step()
        lrs.append(scheduler.get_last_lr()[0])
        steps.append(epoch * BATCHES + batch)

max_lr = max(lrs)
min_lr = min(lrs)
print(f"Maximum LR: {max_lr} | Minimum LR: {min_lr}")
plt.figure()
plt.plot(steps, lrs, label='OneCycle')
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
plt.xlabel("Step")
plt.ylabel("Learning Rate")
plt.show()

# <b><span style='color:#F1A424'>|</span> Loss Function</b><a class='anchor' id='loss'></a> [↑](#top) 

***


In [ ]:
import torch.nn.functional as F

# === Reduction = "mean" ===
criterion = nn.KLDivLoss(reduction="mean")
y_pred = F.log_softmax(torch.randn(6, 2, requires_grad=True), dim=1)
y_true = F.softmax(torch.rand(6, 2), dim=1)
print(f"Predictions: {y_pred}")
print(f"Targets: {y_true}")
output = criterion(y_pred, y_true)
print(f"Output: {output}")

print("\n", "="*100, "\n")

# === Reduction = "batchmean" ===
criterion = nn.KLDivLoss(reduction="batchmean")
y_pred = F.log_softmax(torch.randn(2, 6, requires_grad=True), dim=1)
y_true = F.softmax(torch.rand(2, 6), dim=1)
print(f"Predictions: {y_pred}")
print(f"Targets: {y_true}")
output = criterion(y_pred, y_true)
print(f"Output: {output}")

# <b><span style='color:#F1A424'>|</span> Train and Validation Functions</b><a class='anchor' id='functions'></a> [↑](#top) 

***

In [ ]:
def train_epoch(train_loader, model, optimizer, epoch, scheduler, device):
    """One epoch training pass."""
    model.train()
    criterion = nn.KLDivLoss(reduction="batchmean")
    scaler = torch.cuda.amp.GradScaler(enabled=config.AMP)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    
    # ========== ITERATE OVER TRAIN BATCHES ============
    with tqdm(train_loader, unit="train_batch", desc='Train') as tqdm_train_loader:
        for step, batch in enumerate(tqdm_train_loader):
            X = batch.pop("X").to(device) # send inputs to `device`
            y = batch.pop("y").to(device) # send labels to `device`
            batch_size = y.size(0)
            with torch.cuda.amp.autocast(enabled=config.AMP):
                y_preds = model(X)
                loss = criterion(F.log_softmax(y_preds, dim=1), y)
            if config.GRADIENT_ACCUMULATION_STEPS > 1:
                loss = loss / config.GRADIENT_ACCUMULATION_STEPS
            losses.update(loss.item(), batch_size)
            scaler.scale(loss).backward()
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), config.MAX_GRAD_NORM)
            
            if (step + 1) % config.GRADIENT_ACCUMULATION_STEPS == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                global_step += 1
                scheduler.step()
            end = time.time()

            # ========== LOG INFO ==========
            if step % config.PRINT_FREQ == 0 or step == (len(train_loader)-1):
                print('Epoch: [{0}][{1}/{2}] '
                      'Elapsed {remain:s} '
                      'Loss: {loss.avg:.4f} '
                      'Grad: {grad_norm:.4f}  '
                      'LR: {lr:.8f}  '
                      .format(epoch+1, step, len(train_loader), 
                              remain=timeSince(start, float(step+1)/len(train_loader)),
                              loss=losses,
                              grad_norm=grad_norm,
                              lr=scheduler.get_last_lr()[0]))

    return losses.avg


def valid_epoch(valid_loader, model, device):
    model.eval() 
    softmax = nn.Softmax(dim=1)
    losses = AverageMeter()
    prediction_dict = {}
    preds = []
    start = end = time.time()
    with tqdm(valid_loader, unit="valid_batch", desc='Validation') as tqdm_valid_loader:
        for step, batch in enumerate(tqdm_valid_loader):
            X = batch.pop("X").to(device) 
            y = batch.pop("y").to(device)
            batch_size = y.size(0)
            with torch.no_grad():
                y_preds = model(X)
                loss = criterion(F.log_softmax(y_preds, dim=1), y)
            if config.GRADIENT_ACCUMULATION_STEPS > 1:
                loss = loss / config.GRADIENT_ACCUMULATION_STEPS
            losses.update(loss.item(), batch_size)
            y_preds = softmax(y_preds)
            preds.append(y_preds.to('cpu').numpy()) 
            end = time.time()

            # ========== LOG INFO ==========
            if step % config.PRINT_FREQ == 0 or step == (len(valid_loader)-1):
                print('EVAL: [{0}/{1}] '
                      'Elapsed {remain:s} '
                      'Loss: {loss.avg:.4f} '
                      .format(step, len(valid_loader),
                              remain=timeSince(start, float(step+1)/len(valid_loader)),
                              loss=losses))
                
    prediction_dict["predictions"] = np.concatenate(preds)
    return losses.avg, prediction_dict

# <b><span style='color:#F1A424'>|</span> Train Loop</b><a class='anchor' id='train_loop'></a> [↑](#top) 

***

In [ ]:
def train_loop(df, fold):
    
    #LOGGER.info(f"========== Fold: {fold} training ==========")
    print((f"========== Fold: {fold} training =========="))

    # ======== SPLIT ==========
    train_folds = df[df['fold'] != fold].reset_index(drop=True)
    valid_folds = df[df['fold'] == fold].reset_index(drop=True)
    
    # ======== DATASETS ==========
    train_dataset = CustomDataset(train_folds, config, mode="train")
    valid_dataset = CustomDataset(valid_folds, config, mode="train")
    
    # ======== DATALOADERS ==========
    train_loader = DataLoader(train_dataset,
                              batch_size=config.BATCH_SIZE_TRAIN,
                              shuffle=True,
                              num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=config.BATCH_SIZE_VALID,
                              shuffle=False,
                              num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=False)
    
    # ======== MODEL ==========
    model = CustomModel()
    model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=0.1, weight_decay=config.WEIGHT_DECAY)
    scheduler = OneCycleLR(
        optimizer,
        max_lr=1e-3,
        epochs=config.EPOCHS,
        steps_per_epoch=len(train_loader),
        pct_start=0.1,
        anneal_strategy="cos",
        final_div_factor=100,
    )

    # ======= LOSS ==========
    criterion = nn.KLDivLoss(reduction="batchmean")
    
    best_loss = np.inf
    # ====== ITERATE EPOCHS ========
    for epoch in range(config.EPOCHS):
        start_time = time.time()

        # ======= TRAIN ==========
        avg_train_loss = train_epoch(train_loader, model, optimizer, epoch, scheduler, device)

        # ======= EVALUATION ==========
        avg_val_loss, prediction_dict = valid_epoch(valid_loader, model, device)
        predictions = prediction_dict["predictions"]
        
        # ======= SCORING ==========
        elapsed = time.time() - start_time

        #LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_train_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        print(f'Epoch {epoch+1} - avg_train_loss: {avg_train_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            #LOGGER.info(f'Epoch {epoch+1} - Save Best Loss: {best_loss:.4f} Model')
            print(f'Epoch {epoch+1} - Save Best Loss: {best_loss:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                         f"modelsA4/wavenet_fold_{fold}_best.pth")

    predictions = torch.load(f"modelsA4/wavenet_fold_{fold}_best.pth", 
                             map_location=torch.device('cpu'))['predictions']
    valid_folds[target_preds] = predictions

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

In [ ]:
def train_loop(df, fold, prevmodel = None, best_loss = np.inf):
    
    #LOGGER.info(f"========== Fold: {fold} training ==========")
    print((f"========== Fold: {fold} training =========="))

    # ======== SPLIT ==========
    train_folds = df[df['fold'] != fold].reset_index(drop=True)
    valid_folds = df[df['fold'] == fold].reset_index(drop=True)
    
    # ======== DATASETS ==========
    train_dataset = CustomDataset(train_folds, config, mode="train")
    valid_dataset = CustomDataset(valid_folds, config, mode="train")
    
    # ======== DATALOADERS ==========
    train_loader = DataLoader(train_dataset,
                              batch_size=config.BATCH_SIZE_TRAIN,
                              shuffle=True,
                              num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=config.BATCH_SIZE_VALID,
                              shuffle=False,
                              num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=False)
    
    # ======== MODEL ==========
    model = CustomModel()

    # Move the model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Load the saved state dictionary
    state_dict = torch.load(prevmodel)['model']
    #state_dict = {f"module.{key}": value for key, value in state_dict.items()}

    # Load the modified state dictionary into the model
    model = nn.DataParallel(model)  # Wrap the model with DataParallel
    model.load_state_dict(state_dict)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.1, weight_decay=config.WEIGHT_DECAY)
    scheduler = OneCycleLR(
        optimizer,
        max_lr=1e-3,
        epochs=config.EPOCHS,
        steps_per_epoch=len(train_loader),
        pct_start=0.1,
        anneal_strategy="cos",
        final_div_factor=100,
    )

    # ======= LOSS ==========
    criterion = nn.KLDivLoss(reduction="batchmean")
    
    # ====== ITERATE EPOCHS ========
    for epoch in range(config.EPOCHS):
        start_time = time.time()

        # ======= TRAIN ==========
        avg_train_loss = train_epoch(train_loader, model, optimizer, epoch, scheduler, device)

        # ======= EVALUATION ==========
        avg_val_loss, prediction_dict = valid_epoch(valid_loader, model, device)
        predictions = prediction_dict["predictions"]
        
        # ======= SCORING ==========
        elapsed = time.time() - start_time

        #LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_train_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        print(f'Epoch {epoch+1} - avg_train_loss: {avg_train_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            #LOGGER.info(f'Epoch {epoch+1} - Save Best Loss: {best_loss:.4f} Model')
            print(f'Epoch {epoch+1} - Save Best Loss: {best_loss:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': predictions,
                        'best_loss': best_loss},
                         f"modelsA8/wavenet_fold_{fold}_best.pth")

    predictions = torch.load(f"modelsA8/wavenet_fold_{fold}_best.pth", 
                             map_location=torch.device('cpu'))['predictions']
    valid_folds[target_preds] = predictions

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

# <b><span style='color:#F1A424'>|</span> Train</b><a class='anchor' id='train'></a> [↑](#top) 

***

In [ ]:
def get_result(oof_df):
    kl_loss = nn.KLDivLoss(reduction="batchmean")
    labels = torch.tensor(oof_df[label_cols].values)
    preds = torch.tensor(oof_df[target_preds].values)
    preds = F.log_softmax(preds, dim=1)
    result = kl_loss(preds, labels)
    return result



In [ ]:
#use modelsA6 add to modelsA4
#use models2 and models6

In [ ]:
if not config.TRAIN_FULL_DATA:
        oof_df = pd.DataFrame()
        for fold in range(config.FOLDS):
            if fold in [2,3,4]:
                _oof_df = train_loop(train_df, fold,f"modelsA6/wavenet_fold_{fold}_best.pth")#, torch.load(f"models/wavenet_fold_{fold}_best.pth")["best_loss"])
                oof_df = pd.concat([oof_df, _oof_df])
                print(f"========== Fold {fold} result: {get_result(_oof_df)} ==========")
                #LOGGER.info(f"========== Fold {fold} finished ==========")
        oof_df = oof_df.reset_index(drop=True)
        oof_df.to_csv('modelsA8/oof_df.csv', index=False)
else:
    train_loop_full_data(train_df)

In [28]:
        oof_df = oof_df.reset_index(drop=True)
        oof_df.to_csv('modelsA8/oof_df.csv', index=False)

In [50]:
!mkdir modelsA8

In [35]:
if not config.TRAIN_FULL_DATA:
        oof_df = pd.DataFrame()
        for fold in range(config.FOLDS):
            if fold in [0,1,2,3,4]:
                _oof_df = train_loop(train_df, fold,f"models/wavenet_fold_{fold}_best.pth")#, torch.load(f"models/wavenet_fold_{fold}_best.pth")["best_loss"])
                oof_df = pd.concat([oof_df, _oof_df])
                print(f"========== Fold {fold} result: {get_result(_oof_df)} ==========")
                #LOGGER.info(f"========== Fold {fold} finished ==========")
        oof_df = oof_df.reset_index(drop=True)
        oof_df.to_csv('models3/oof_df.csv', index=False)
else:
    train_loop_full_data(train_df)

========== Fold: 0 training ==========


Train:   0%|          | 0/60 [00:00<?, ?train_batch/s]/home/mccruz/.local/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Train:   2%|▏         | 1/60 [00:02<01:59,  2.03s/train_batch]

Epoch: [1][0/60] Elapsed 0m 2s (remain 1m 59s) Loss: 1.5122 Grad: inf  LR: 0.00004017  


Train:  85%|████████▌ | 51/60 [00:23<00:03,  2.35train_batch/s]

Epoch: [1][50/60] Elapsed 0m 23s (remain 0m 4s) Loss: 1.6079 Grad: 117629.7109  LR: 0.00041319  


...

Epoch: [20][59/60] Elapsed 0m 24s (remain 0m 0s) Loss: 0.0660 Grad: 46149.2969  LR: 0.00000040  


Validation:   5%|▍         | 1/21 [00:00<00:10,  1.86valid_batch/s]

EVAL: [0/21] Elapsed 0m 0s (remain 0m 10s) Loss: 0.4175 


Validation: 100%|██████████| 21/21 [00:04<00:00,  4.30valid_batch/s]


EVAL: [20/21] Elapsed 0m 4s (remain 0m 0s) Loss: 0.2816 
Epoch 20 - avg_train_loss: 0.0660  avg_val_loss: 0.2816  time: 29s
Epoch 20 - Save Best Loss: 0.2816 Model
========== Fold 3 result: 0.9548029344691641 ==========


In [27]:
if not config.TRAIN_FULL_DATA:
        oof_df = pd.DataFrame()
        for fold in range(config.FOLDS):
            if fold in [3,4]:
                _oof_df = train_loop(train_df, fold,f"models3/wavenet_fold_{fold}_best.pth")#, torch.load(f"models/wavenet_fold_{fold}_best.pth")["best_loss"])
                oof_df = pd.concat([oof_df, _oof_df])
                print(f"========== Fold {fold} result: {get_result(_oof_df)} ==========")
                #LOGGER.info(f"========== Fold {fold} finished ==========")
        oof_df = oof_df.reset_index(drop=True)
        oof_df.to_csv('models5/oof_df.csv', index=False)
else:
    train_loop_full_data(train_df)

========== Fold: 3 training ==========


Train:   0%|          | 0/119 [00:00<?, ?train_batch/s]/home/mccruz/.local/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Train:   1%|          | 1/119 [00:13<27:03, 13.76s/train_batch]

Epoch: [1][0/119] Elapsed 0m 13s (remain 27m 3s) Loss: 2.1429 Grad: inf  LR: 0.00004004  


Train:  43%|████▎     | 51/119 [00:24<00:14,  4.67train_batch/s]

Epoch: [1][50/119] Elapsed 0m 24s (remain 0m 33s) Loss: 1.8200 Grad: 69308.2734  LR: 0.00014557  


...

EVAL: [0/40] Elapsed 0m 0s (remain 0m 26s) Loss: 0.3726 


Validation: 100%|██████████| 40/40 [00:09<00:00,  4.22valid_batch/s]

EVAL: [39/40] Elapsed 0m 9s (remain 0m 0s) Loss: 0.2623 


Validation: 100%|██████████| 40/40 [00:10<00:00,  3.90valid_batch/s]


Epoch 20 - avg_train_loss: 0.0710  avg_val_loss: 0.2623  time: 60s
========== Fold 3 result: 0.9395570480745696 ==========


In [29]:
get_result(oof_df)

tensor(0.9396, dtype=torch.float64)